## Introduction

This project processes and analyzes historical data for video game sales up to the year 2016. The scope of the analysis for this project is to determine any patterns that could assist an advertisement campaign that would improve profitability of video game sales for the following year. Additionally, we'll run two hypothesis tests (population means) to aid our decision on what recommendations we should make as a result of our analysis.

The dataset is provided in the follow file:

**games.csv**
- *Name*: game title
- *Platform*: platforms of release
- *Year_of_Release*: Year game was released on specified platform
- *Genre*: game genre
- *NA_sales*: units sold in North America (in millions)
- *EU_sales*: units sold in Europe (in millions)
- *JP_sales*: units sold in Japan (in millions)
- *Other_sales* units sold in other regions (in millions)
- *Critic_Score*: profession game critic score (out of 100)
- *User_Score*: user score (out of 10)
- *Rating*: Entertainment Software Rating Board (ESRB) rating (https://www.esrb.org/ratings-guide/)

## Initialization

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats as st

**Reading the Dataset**

Setting the display options and reading the csv file into the DataFrame, df_games.

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)

# When cloning this project, ensure directory/file paths are correct with respect to user's operating system
df_games = pd.read_csv('datasets/games.csv')

Displaying general information and a random sample of the data.

In [3]:
df_games.info()
display(df_games.sample(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
7616,Clive Barker's Jericho,PS3,2007.0,Shooter,0.17,0.01,0.00,0.02,60.0,7.1,M
11738,Mind Quiz: Your Brain Coach,DS,2006.0,Misc,0.00,0.00,0.08,0.00,NaN,tbd,E
1862,NBA Live 2001,PS2,2001.0,Sports,0.52,0.40,0.04,0.13,74.0,tbd,E
7837,Chou-Kuukan Night Pro Yakyuu King (weekly JP sales),N64,1996.0,Sports,0.00,0.00,0.19,0.00,NaN,NaN,NaN
5793,Pro Evolution Soccer 2013,X360,2012.0,Sports,0.08,0.20,0.00,0.03,82.0,6.9,E
7525,The Bible Game,PS2,2005.0,Misc,0.10,0.08,0.00,0.03,NaN,5.4,E
1691,Captain Toad: Treasure Tracker,WiiU,2014.0,Puzzle,0.53,0.38,0.19,0.09,81.0,8.6,E
7882,3rd Super Robot Wars Z: Tengoku-Hen,PSV,2015.0,Action,0.00,0.00,0.19,0.00,NaN,NaN,NaN
13493,Danny Phantom: Urban Jungle,DS,2006.0,Action,0.04,0.00,0.00,0.00,NaN,NaN,NaN
1426,Centipede,2600,1981.0,Shooter,1.26,0.08,0.00,0.01,NaN,NaN,NaN


## Preparing the Data ##

Print out of total missing values by column.

In [4]:
print(df_games.isna().sum())

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
NA_sales              0
EU_sales              0
JP_sales              0
Other_sales           0
Critic_Score       8578
User_Score         6701
Rating             6766
dtype: int64



**Column Names, Missing Titles, and Missing Genre**

As a first step, we'll convert the column names to snake_case to faciliate column referencing. We can see the dataset has missing values in multiple columns. This data could be missing for multiple reasons such as information from older platforms being unavailable, sales records for games being stored on multiple rows for different regions, or just incomplete data collection during certain years. When considering how to fill in the missing values, each column will be handled individually in order to use appropriate replacements as to mitigate adverse calculations during follow-on analysis.

There are only two games with missing names and genre. The information provided by these two entries is useless without any identifiers, thus they will be dropped from the DataFrame.

In [5]:
df_games.rename(str.lower, axis=1, inplace=True)
df_games = df_games.dropna(how='all', subset=['name', 'genre'])

print('Number of missing names: ', df_games['name'].isna().sum())
print('Number of missing genres: ', df_games['genre'].isna().sum())

Number of missing names:  0
Number of missing genres:  0


**Missing Years**  
 
Since there are only a few entries with missing years (less than 2%), we'll elect to just drop those rows, then type cast the column to integer.

In [6]:
df_games = df_games.dropna(subset=['year_of_release'])
df_games['year_of_release'] = df_games['year_of_release'].astype(int)
print('Number of missing years: ', df_games['year_of_release'].isna().sum())

Number of missing years:  0


**Filling in Missing Ratings**

We will assign 'unknown', to missing ratings rather than trying to intuit a suitable replacement. As not every region may use the ESRB system, it may be more useful in analysis to track the gaps of missing values instead of assigning a rating. Additionally, we'll update any of the depricated ratings with the more current label. We'll print out a list of the unique ratings and sum of missing values in the rating column to verify changes.

In [7]:
df_games['rating'] = df_games['rating'].fillna('unknown')
df_games['rating'] = df_games['rating'].replace(['EC', 'K-A'], 'E')

print(df_games['rating'].unique())
print('Number of missing ratings: ', df_games['rating'].isna().sum())

['E' 'unknown' 'M' 'T' 'E10+' 'AO' 'RP']
Number of missing ratings:  0


**Filling in Missing Scores**

Since there is a large number of missing scores, restoring them and maintaining an accurate analysis is difficult. As such they will be left as NaN. In the scope of this project's analysis, 'tbd' and 'NaN' are effectively the same, thus 'tbd' will be changed to NaN to facilitate the use of certain methods later on.

We'll verify changes by checking the new sums of missing values in each column, but we're expecting some NaN values to remain.

In [8]:
df_games['user_score'].replace('tbd', None, inplace=True)
df_games['user_score'] = df_games['user_score'].astype(float)

print('Number of missing critic scores: ', df_games['critic_score'].isna().sum())
print('NUmber of missing user scores: ', df_games['user_score'].isna().sum())

Number of missing critic scores:  8461
NUmber of missing user scores:  8981


**Removing Duplicate Values**

In [9]:
print('Number of exact duplicates: ', df_games.duplicated().sum())

Number of exact duplicates:  0


There are no explicit duplicates. In terms of implicit duplicates, we can examine the dataset to see if there are multiple entries for what should be the same game. For example, there could be multiple entries for a title that was released on a platform, but their regional sales were recorded separately. We'll define duplicates if there are entries with the same game name, platform, year of release, critic score and user score. For the case of the 'unknown' game, it is possible that these are two different titles, however combing the data is neglible for our analysis.

In [10]:
dupes_mask = df_games.duplicated(subset=['name', 'platform', 'year_of_release', 'critic_score', 'user_score'], keep=False)
display(df_games[dupes_mask].sort_values('name'))

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
604,Madden NFL 13,PS3,2012,Sports,2.11,0.22,0.0,0.23,83.0,5.5,E
16230,Madden NFL 13,PS3,2012,Sports,0.00,0.01,0.0,0.00,83.0,5.5,E


There are only a few dupicates matching our criteria. We can combine the sales data by applying an aggregate function to sum the respective columns, and then replace the duplicates using the where( ) method. The result will create explicit duplicates, which we can then drop using drop_duplicates( ). After the operation we can verify changes by printing the sum of duplicated values.

In [11]:
dupes = df_games[dupes_mask].groupby('name').agg({'na_sales': 'sum', 'eu_sales': 'sum', 'jp_sales': 'sum', 'other_sales': 'sum'}).reset_index()

for index, rows in dupes.iterrows():
    df_games['na_sales'] = df_games['na_sales'].where(~((dupes_mask) & (df_games['name'] == rows['name'])), rows['na_sales'])
    df_games['eu_sales'] = df_games['eu_sales'].where(~((dupes_mask) & (df_games['name'] == rows['name'])), rows['eu_sales'])
    df_games['jp_sales'] = df_games['jp_sales'].where(~((dupes_mask) & (df_games['name'] == rows['name'])), rows['jp_sales'])
    df_games['other_sales'] = df_games['other_sales'].where(~((dupes_mask) & (df_games['name'] == rows['name'])), rows['other_sales'])

df_games.drop_duplicates(inplace=True)
print('Number of exact duplicates: ', df_games.duplicated().sum())
print('Number of implicit duplicates: ', df_games.duplicated(subset=['name', 'platform', 'year_of_release', 'critic_score', 'user_score']).sum())

Number of exact duplicates:  0
Number of implicit duplicates:  0


**Adding New Column: Total Sales**

Totaling sales for each region and displaying a sample of the newly processed dataset.

In [12]:
df_games['total_sales'] = df_games[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)
display(df_games.sample(10))

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
5748,Brothers In Arms: Earned in Blood,XB,2005,Shooter,0.23,0.07,0.00,0.01,85.0,8.6,M,0.31
6935,NASCAR 08,PS2,2007,Racing,0.12,0.09,0.00,0.03,56.0,8.2,E,0.24
8628,Solitaire Overload Plus,DS,2010,Misc,0.15,0.00,0.00,0.01,NaN,NaN,E,0.16
15651,Miyako,PSP,2010,Adventure,0.00,0.00,0.02,0.00,NaN,NaN,unknown,0.02
14512,Gotouchi Tetsudou: Gotouchi Chara to Nihon Zenkoku no Tabi,WiiU,2014,Misc,0.00,0.00,0.03,0.00,NaN,NaN,unknown,0.03
5181,NARC,PS2,2005,Shooter,0.18,0.14,0.00,0.05,50.0,7.0,M,0.37
16486,Super Dungeon Bros,PS4,2016,Action,0.01,0.00,0.00,0.00,42.0,2.3,E10+,0.01
1632,Need for Speed Carbon,PS3,2006,Racing,0.49,0.50,0.03,0.20,75.0,7.3,E10+,1.22
7977,Phantasy Star Online Ver. 2,DC,2001,Role-Playing,0.00,0.00,0.19,0.00,80.0,8.9,T,0.19
2866,Ben 10: Protector of Earth,PSP,2007,Action,0.23,0.30,0.00,0.18,60.0,7.2,E10+,0.71


## Analyzing the Data ##

**Number of Games Released Each Year**

In [13]:
releases_by_year = df_games.groupby('year_of_release')['name'].count().reset_index(name='total_games')
display(releases_by_year)

fig = px.bar(releases_by_year, 
            x='year_of_release',
            y='total_games',
            labels={'year_of_release': 'Year', 'total_games': 'Number of Games Released'},
            title='Games Released by Year')
fig.show()

,year_of_release,total_games
0,1980,9
1,1981,46
2,1982,36
3,1983,17
4,1984,14
5,1985,14
6,1986,21
7,1987,16
8,1988,15
9,1989,17


Generally, the data shows an increase in game releases over time. Furthermore, there is a significant increase in game production after 1993. A possible explanation for this trend could be an increasing popularity of video games as platforms became more accessible and technologically advanced. For example, the release dates of the Xbox (2001) and PlayStation 2 (2000) consoles were followed by a markable increase in video game releases. 

The number of video game releases appears to peak in the late 2000s and then shows a significant drop in game production in the following years. While a decrease of video game popularity is a possibility, this decline could also be due to modern platforms having backwards compatibility or increased replayability (i.e. game's lifespan) for their games.

**Sales Performance by Platform**

In [14]:
sales_by_platform = df_games.groupby('platform')['total_sales'].sum().reset_index(name='total_sales_to_date').sort_values('total_sales_to_date', ascending=False)
display(sales_by_platform)

fig = px.bar(sales_by_platform, x='platform',
            y='total_sales_to_date',
            labels={'platform': 'Platform', 'total_sales_to_date': 'Total Games Sold (Millions)'},
            title='Total Sales to Date by Platform')
fig.show()

,platform,total_sales_to_date
16,PS2,1233.56
28,X360,961.24
17,PS3,931.34
26,Wii,891.18
4,DS,802.78
15,PS,727.58
18,PS4,314.14
6,GBA,312.88
19,PSP,289.53
2,3DS,257.81


According to the data, the PlayStation 2 (PS2) is the most popular platform with over 1.2 billion titles sold as of 2016. If we look at the top six platforms, their popularity is higher than the other platforms by a significant margin. As such, we'll build a distribution of yearly sales for the top six platforms. *Note: The data shows the Nintendo DS had their earliest game release in 1985, but the platform was not released until 2004 (which matches the rest of the release years for the DS). This is the only data point outside of the pertinent range for analysis. As such, we'll filter out this data point when analyzing the sales performance for the top six platforms.*

**Yearly Sales Performance - Top Six Platforms**

Line plot of the top performing platforms overlayed in the same plot. Individual platforms can be toggled using the legend on the right to see individual sales performance, sales comparisons, etc.

In [15]:
df_top_platforms = df_games.query('platform in @sales_by_platform.platform.head(6) and year_of_release > 1985')
yearly_sales_top_platform = df_top_platforms.groupby(['year_of_release', 'platform'])['total_sales'].sum().reset_index()

fig = px.line(yearly_sales_top_platform,
            x='year_of_release',
            y='total_sales',
            color='platform',
            markers=True,
            labels={'year_of_release': 'Year', 'total_sales': 'Total Games Sold (Millions)'},
            title='Yearly Sales Performance - Top Six Platforms')
fig.show()

In [16]:
display(df_top_platforms.groupby('platform')['year_of_release'].agg(['min', 'max', np.ptp]).sort_values('ptp', ascending=False))

,min,max,ptp
platform,,,
PS2,2000,2011,11
X360,2005,2016,11
PS3,2006,2016,10
Wii,2006,2016,10
DS,2004,2013,9
PS,1994,2003,9


The information above shows that the PlayStation, PlayStation 2, and Nintendo DS were among the most popular platforms, but no longer have sales. This is likely due to manufacturers releasing newer generations of those platforms and/or newer games not being released on depricated systems. The peak-to-peak (ptp) column shows the lifespans for these three platforms are about a decade (9 years for PS and DS, 11 years for PS2).

In order to identify patterns that may improve profitabily in 2017, we can track sales performance beginning in 2012. During this period of time, we'll see the tail end of the most successful platforms and the current trends for video game sales on modern platforms.

**Yearly Sales Performance: 2012 - 2016**

Line plot of yearly sales performance beginning in 2012. Individual platforms can be toggled using the legend on the right to see individual sales performance, sales comparisons, etc.

In [17]:
filtered_years = df_games[df_games['year_of_release'] >= 2012]
yearly_sales_by_platform = filtered_years.groupby(['year_of_release', 'platform'])['total_sales'].sum().reset_index()

fig = px.line(yearly_sales_by_platform,
            x='year_of_release',
            y='total_sales',
            color='platform',
            markers=True,
            hover_data=['platform', 'total_sales'],
            labels={'year_of_release': 'Year', 'total_sales': 'Total Games Sold (Millions)'},
            title='Year Sales Performance: 2012-2016',
            height=600)
fig.update_xaxes(dtick=1)
fig.show()

Currently, the leader in sales are the PlayStation 4, Xbox One, and 3DS. All of the platforms are shrinking in sales, however, this could be due to incomplete data for 2016. The Xbox One and PS4 were the only two platforms that had growth in the previous years. As such, advertisements should target PS4 and Xbox One as potentially profitable platforms in 2017.


**Box Plots of Global Game Sales: Xbox One vs. PlayStation 4**
- Plotted horizontally for readability.
- Q1, Q3, and whiskers calculated using default method (linear).
- Platforms can be toggled on/off
- Zoom in on plot via cursor

Also displaying average, quartiles, min, max, and median.

In [18]:
potential_platforms_extended = ['PS4', 'XOne', '3DS', 'WiiU', 'PC', 'PSV']

potential_platforms_data = filtered_years.query('platform.isin(@potential_platforms_extended)')
                                                
fig = px.box(potential_platforms_data,
                y='platform',   
                x='total_sales',
                orientation='h',
                color='platform',
                labels={'platform': 'Platform', 'total_sales': 'Number of Copies Sold (Millions)'},
                title='Global Game Sales by Platform')

fig.update_layout(legend=dict(orientation='h', yanchor='bottom', y=1.02,
                              xanchor='left', x=.001,
                              title=None)
                 )
fig.show()

display(potential_platforms_data.groupby('platform')['total_sales'].describe().sort_values('mean', ascending=False))

,count,mean,std,min,25%,50%,75%,max
platform,,,,,,,,
PS4,392.0,0.801378,1.609456,0.01,0.06,0.20,0.7300,14.63
XOne,247.0,0.645020,1.036139,0.01,0.06,0.22,0.6850,7.39
WiiU,147.0,0.559116,1.058836,0.01,0.08,0.22,0.5250,7.09
3DS,396.0,0.491439,1.387161,0.01,0.04,0.11,0.3125,14.60
PC,250.0,0.250600,0.494640,0.01,0.03,0.08,0.2400,5.14
PSV,411.0,0.119659,0.203011,0.01,0.02,0.05,0.1200,1.96


Overall, the Xbox One and PlayStation 4 have similar sales performance in terms of median and their quartile values and the average for both platforms is higher than their median. There are a few outliers for both platforms; while platform populartiy could be a factor, this could also be due to in part by platform exclusives.

**Correlation of Professional Reviews to Sales: PlayStation 3**

Scatter plot of critic score (y-axis) against total sales (x-axis) for games on the PS3, and Pearson correlation coefficient.

In [19]:
ps3_data = filtered_years[filtered_years['platform'] == 'PS3']

fig1 = px.scatter(ps3_data,
                x='total_sales',
                y='critic_score',
                labels={'critic_score': 'Score (out of 100)',
                        'total_sales': 'Number of Copies Sold (Millions)'},
                title='Critic Score vs. Total Sales')
fig2 = px.scatter(ps3_data,
                x='total_sales',
                y='user_score',
                labels={'user_score': 'Score (out of 10)',
                        'total_sales': 'Number of Copies Sold (Millions)'},
                title='User Score vs. Total Sales')                
fig1.show()
fig2.show()

print('Pearson Coefficient (Sales to Critic Score): ', ps3_data['total_sales'].corr(ps3_data['critic_score']))
print('Pearson Coefficient (Sales to User Score): ', ps3_data['total_sales'].corr(ps3_data['user_score']))

Pearson Coefficient (Sales to Critic Score):  0.33416558787023776
Pearson Coefficient (Sales to User Score):  -0.006217851960559287


According to the first scatter plot and Pearson correlation coefficient, there is a moderately positive correlation between professional reviews scores and game sales. While it is logical to believe the platform's best selling game has a high review score, it is interesting to note there are many games with very positive reviews (e.g. > 80) and relatively poor sales performance (e.g. < 1M global sales).

When comparing user score and total sales, the data shows there is almost no correlation (slightly negative Pearson coefficient) between the two. Similarily, the best seller has a high user score, and there are many games with a high score and poor sales performance. There are even a few games with high sales, but a relatively poor user score.

Overall, this also shows that game reviews aren't the driving factor for a game's success.

**Critic/User Review Affect on Sales Performance: Xbox One and PS4**

Scatter plot of review scores to total sales with Pearson coefficients.

In [20]:
fig1 = px.scatter(potential_platforms_data,
                x='total_sales',
                y='critic_score',
                color='platform',
                labels={'critic_score': 'Score (out of 100)',
                        'total_sales': 'Number of Copies Sold (Millions)'},
                title='Critic Score vs. Total Sales')
                
fig2 = px.scatter(potential_platforms_data,
                x='total_sales',
                y='user_score',
                color='platform',
                labels={'user_score': 'Score (out of 10)',
                        'total_sales': 'Number of Copies Sold (Millions)'},
                title='User Score vs. Total Sales')                
fig1.show()
for plat, group in potential_platforms_data.groupby('platform'):
        print('Pearson Coefficient (Sales to Critic Score) on ', plat, ': ', group['total_sales'].corr(group['critic_score']))

fig2.show()
for plat, group in potential_platforms_data.groupby('platform'):
        print('Pearson Coefficient (Sales to User Score) on ', plat, ': ', group['total_sales'].corr(group['user_score']))

Pearson Coefficient (Sales to Critic Score) on  3DS :  0.32080286953607845
Pearson Coefficient (Sales to Critic Score) on  PC :  0.2372432431156793
Pearson Coefficient (Sales to Critic Score) on  PS4 :  0.4065679020617816
Pearson Coefficient (Sales to Critic Score) on  PSV :  0.09448789929316172
Pearson Coefficient (Sales to Critic Score) on  WiiU :  0.3483797601201749
Pearson Coefficient (Sales to Critic Score) on  XOne :  0.41699832800840164


Pearson Coefficient (Sales to User Score) on  3DS :  0.19758263411038596
Pearson Coefficient (Sales to User Score) on  PC :  -0.12186747630735582
Pearson Coefficient (Sales to User Score) on  PS4 :  -0.03195711020455643
Pearson Coefficient (Sales to User Score) on  PSV :  -0.0043394375143148665
Pearson Coefficient (Sales to User Score) on  WiiU :  0.4002190202148617
Pearson Coefficient (Sales to User Score) on  XOne :  -0.06892505328279415


When looking at the potentially profitable platforms, we see the previously observed trend for a game review's affect on sales is still valid for the most part. The 3DS, PC, PS4, Wii U, and Xbox One have a slightly to moderately positive Pearson coefficient when comparing professional reviews to total game sales. The PSV has almost no correlation (a weak positive correlation) in sales to professional reviews.

There is almost no correlation in sales to user score for the PSV, PS4, and Xbox One. There is, however, a moderately positive correlation in sales to user score for the Wii U and slightly positive correlation for the 3DS. The PC has slightly weak negative correlation in sales to user score.

**Distribution of Games by Genre**

Plotting a histogram for game distribution by genre. Also, displaying a boxplot and aggregate sales data for games grouped by genre.

In [21]:
fig1 = px.histogram(filtered_years, x= 'genre', title='Distribution of Games by Genre')
fig2 = px.box(filtered_years,
            x='total_sales',   
            y='genre',
            orientation='h',
            labels={'genre': 'Genre', 'total_sales': 'Number of Copies Sold (Millions)'},
            title='Global Game Sales by Genre',
            height=600)
fig1.show()
fig2.show()

sales_info_by_genre =filtered_years.groupby('genre')['total_sales'].describe().reset_index()

print('Genres sorted by distribution (High to low):')
display(sales_info_by_genre[['genre', 'count']].sort_values('count', ascending=False))

print('Top and bottom 3 genres sorted by average:')
display(sales_info_by_genre[['genre', 'mean']].sort_values('mean', ascending=False).head(3))
display(sales_info_by_genre[['genre', 'mean']].sort_values('mean', ascending=False).tail(3))

print('Top and bottom 3 genres sorted by median:')
display(sales_info_by_genre[['genre', '50%']].sort_values('50%', ascending=False).head(3))
display(sales_info_by_genre[['genre', '50%']].sort_values('50%', ascending=False).tail(3))

Genres sorted by distribution (High to low):


,genre,count
0,Action,1031.0
7,Role-Playing,370.0
1,Adventure,302.0
10,Sports,267.0
8,Shooter,235.0
3,Misc,192.0
6,Racing,115.0
2,Fighting,109.0
4,Platform,85.0
9,Simulation,80.0


Top and bottom 3 genres sorted by average:


,genre,mean
8,Shooter,1.296723
4,Platform,0.717647
10,Sports,0.678165


,genre,mean
11,Strategy,0.187887
5,Puzzle,0.174643
1,Adventure,0.097450


Top and bottom 3 genres sorted by median:


,genre,50%
8,Shooter,0.44
10,Sports,0.24
4,Platform,0.21


,genre,50%
11,Strategy,0.080
5,Puzzle,0.045
1,Adventure,0.030


According to the data, the most common genres are not necessarily the most profitable. In fact, the most profitable genres (Shooters, Sports, and Platformers) have a modest share of the number of titles when compared to the most common genre, Action. Futhermore, Action genre games consistently perform below the aforemented top three. The least profitable games appear to be Strategy, Puzzle, and Adventure. With the exception of Adventure, these genres have the smallest share in terms of the number of titles.

## Creating a User Profile by Region: NA, EU, JP ##

We define the function, ***get_top_five***, to determine the top five sales performers in a region according to a given category.

***Parameters***  
- *data*: DataFrame of video game sales.
- *category*: Column name (str) on which the data will be grouped.
- *region*: Column name (str) of the region by which the sales aggregate calculations will be performed (i.e. 'na_sales', 'eu_sales', or 'jp_sales')

***Returns***: The top five sales performers by category for the specified region ranked by total sales. Also returns the market share by category.

In [22]:
def get_top_five(data, category, region):
    df = data.groupby(category)[region].agg(['sum', 'mean']).reset_index()
    df['market_share'] = df['sum']/data[region].sum()

    return df.sort_values('mean', ascending=False).head(5)

**Top Five Platforms by Region**

Looking at the top five platform for North America, Europe, and Japan according to average sales.

In [23]:
na_platforms = get_top_five(filtered_years, 'platform', 'na_sales')
fig = px.bar(na_platforms,
            x='platform',
            y='mean',
            width=800)
fig.show()
display(na_platforms)

eu_platforms = get_top_five(filtered_years, 'platform', 'eu_sales')
fig = px.bar(eu_platforms,
            x='platform',
            y='mean',
            width=800)
fig.show()
display(eu_platforms)

jp_platforms = get_top_five(filtered_years, 'platform', 'jp_sales')
fig = px.bar(jp_platforms,
            x='platform',
            y='mean',
            width=800)
fig.show()
display(jp_platforms)

,platform,sum,mean,market_share
9,X360,140.05,0.479623,0.236983
10,XOne,93.12,0.377004,0.157571
7,Wii,17.45,0.323148,0.029528
4,PS4,108.74,0.277398,0.184003
8,WiiU,38.10,0.259184,0.064470


,platform,sum,mean,market_share
4,PS4,141.09,0.359923,0.278383
9,X360,74.52,0.255205,0.147034
7,Wii,11.92,0.220741,0.023519
3,PS3,106.86,0.217195,0.210844
10,XOne,51.59,0.208866,0.101792


,platform,sum,mean,market_share
0,3DS,87.79,0.221692,0.455862
1,DS,3.72,0.120000,0.019317
8,WiiU,13.01,0.088503,0.067556
3,PS3,35.29,0.071728,0.183249
7,Wii,3.39,0.062778,0.017603


Looing at the top five platforms in North America by average sales, Xbox 360 has the highest average sales and largest market share at about 23.7%. The Xbox One and PS4 share the next highest market share (approximately 15.8% and 18.4%, respectively), but Xbox One has higher average sales by approximately 100,000 units sold per game. The Wii and Wii hold the lowest market shares in the top five platforms (approximately 3% and 6.4%, respectively). Their average sales are comprobable to that of PS4, differing by about 20-50 thousand units sold per game, on average.

In Europe, the PS4 is at the top of the list in both average sales and market share. The remaining platforms have comprobable average sales (between 200 and 250 thousand units sold per game), but their market shares highly vary. For example, the PS3 has the second highest market share (21.1%), but one of the lowest average sales. Conversely, the Wii has the third highest average, but smallest market share.

In Japan, the hand-held platform, 3DS, has the highest average sales. Furthermore, it has the largest market share by a significant margin (approximately 45.6%). Similar to Europe, the remaining platforms in Japan have comprobable average sales, but highly varied market shares. The DS has the second highest average sales, but lowest market share. The PS3 has the second highest market share, but one of the lowest average sales.

In terms of the top platform in each region, the average sales and market shares show there appears to be a platform preference in each region.

**Top Five Genres by Region**

Looking at the top five genres for North America, Europe, and Japan according to average sales.

In [24]:
na_genres = get_top_five(filtered_years, 'genre', 'na_sales')
fig = px.bar(na_genres,
            x='genre',
            y='mean',
            width=800)
fig.show()
display(na_genres)

eu_genres = get_top_five(filtered_years, 'genre', 'eu_sales')
fig = px.bar(eu_genres,
            x='genre',
            y='mean',
            width=800)
fig.show()
display(eu_genres)

jp_genres = get_top_five(filtered_years, 'genre', 'jp_sales')
fig = px.bar(jp_genres,
            x='genre',
            y='mean',
            width=800)
fig.show()
display(jp_genres)

,genre,sum,mean,market_share
8,Shooter,144.77,0.616043,0.244970
10,Sports,81.53,0.305356,0.137960
4,Platform,25.38,0.298588,0.042946
3,Misc,38.19,0.198906,0.064623
2,Fighting,19.79,0.181560,0.033487


,genre,sum,mean,market_share
8,Shooter,113.47,0.482851,0.223886
10,Sports,69.09,0.258764,0.136321
4,Platform,21.41,0.251882,0.042244
6,Racing,27.29,0.237304,0.053846
9,Simulation,14.55,0.181875,0.028708


,genre,sum,mean,market_share
7,Role-Playing,65.44,0.176865,0.339807
9,Simulation,10.41,0.130125,0.054055
4,Platform,8.63,0.101529,0.044813
2,Fighting,9.44,0.086606,0.049019
5,Puzzle,2.14,0.076429,0.011112


Generally speaking, the top genres in each region also have the top market shares (slight variation in the 3rd and 4th ranked genre in each region). Furthermore, North America and Europe appear to have similar preferences in genre. Specifically, the top three genres in sales and market share for these regions are Shooters, Sports, and Platformers. Japan differs in that the top three genres are RPG, Simulation, and then Platfomers.

**ESRB Rating Affect on Sales by Region**

Looking at the top five game ratings for North America, Europe, and Japan according to average sales.

In [25]:
na_ratings = get_top_five(filtered_years, 'rating', 'na_sales')
fig = px.bar(na_ratings,
            x='rating',
            y='mean',
            width=800)
fig.show()
display(na_ratings)

eu_ratings = get_top_five(filtered_years, 'rating', 'eu_sales')
fig = px.bar(eu_ratings,
            x='rating',
            y='mean',
            width=800)
fig.show()
display(eu_ratings)

jp_ratings = get_top_five(filtered_years, 'rating', 'jp_sales')
fig = px.bar(jp_ratings,
            x='rating',
            y='mean',
            width=800)
fig.show()
display(jp_ratings)

,rating,sum,mean,market_share
2,M,231.57,0.465000,0.391847
0,E,114.37,0.290279,0.193529
1,E10+,75.70,0.247386,0.128094
3,T,66.02,0.160243,0.111715
4,unknown,103.31,0.081027,0.174814


,rating,sum,mean,market_share
2,M,193.96,0.389478,0.382700
0,E,113.03,0.286878,0.223018
1,E10+,55.37,0.180948,0.109250
3,T,52.96,0.128544,0.104495
4,unknown,91.50,0.071765,0.180537


,rating,sum,mean,market_share
4,unknown,108.84,0.085365,0.565168
0,E,28.33,0.071904,0.147108
3,T,26.02,0.063155,0.135113
2,M,21.20,0.042570,0.110084
1,E10+,8.19,0.026765,0.042528


For the most part, ESRB rating does not appear to affect sales by region in that North America and Europe have similar average sales performance and market share when delineated by rating. In Japan, a rating of 'unknown' has over 50% of the market share. This could be due to the region not using the ESRB rating system ('unknown' formerly being NaN), in which case, the same conclusion still holds true for this region (i.e. ESRB does not affect sales in Japan). In the case that this region does use ESRB, then the differences in sales based on rating could be due to the accessibility of more restictive titles.

## Testing Hypotheses ##

We'll conduct two hypothesis tests between two different population samples. We'll first look at population means between two platforms, then population means between two genres. The statistical significance in either test need not be stringent, thus both tests will have an alpha value of 5%. Since we're testing statistical means of two sample populations, we'll use the ttest_ind( ) method. This method tests whether the means are the same, thus the null hypothesis for the first example will be a two-tailed tests on whether the average user scores for the given category are the same.

**Null Hypothesis 1:** The average user rating of the Xbox One and PC are the same.  
**Alternative Hypothesis 1:** The average user rating for Xbox One and PC are different.

In [26]:
pc_xone_data = filtered_years[(filtered_years['platform'] == 'XOne') | (filtered_years['platform'] == 'PC')].dropna()
fig = px.histogram(pc_xone_data, x=pc_xone_data['user_score'], color='platform', barmode='overlay')
fig.show()

Since the dataset is skewed and the sample sizes are quite different, we'll first run a Levene test for variance, centered at the median (for skewed datasets) in order to set the appropriate parameters when testing our null hypothesis. 

In [27]:
sample_1 = filtered_years[filtered_years['platform'] == 'XOne']['user_score'].dropna()
sample_2 = filtered_years[filtered_years['platform'] == 'PC']['user_score'].dropna()

print('Levene p-value: ', st.levene(sample_1, sample_2, center='median').pvalue)


Levene p-value:  0.012514842818505541


The Levene test p-value is significantly higher than 1%, indicating we have met the assumption of homogeneity in variance. As such, the equal_var( ) parameter will be set to True.

In [28]:
alpha = 0.05
results = st.ttest_ind(sample_1, sample_2, equal_var = True)
                             
print('T-test p-value: ', results.pvalue)

# if the p-value is less than alpha, we reject the hypothesis
if results.pvalue < alpha:
    print("We reject the null hypothesis")
else:
    print("We can't reject the null hypothesis")

T-test p-value:  0.5535080500643661
We can't reject the null hypothesis


The resulting p-value is over 5%, thus we cannot reject the null hypothesis. This indicates there is enough data in the sample populations to suggest the average user scores are the same from Xbox One and PC.

**Null Hypothesis 2:** The average user rating for Action and Sports genres are the same.  
**Alternative Hypothesis 2:** The average user rating for Action and Sports genres are different.

In [29]:
genre_data = filtered_years[(filtered_years['genre'] == 'Action') | (filtered_years['genre'] == 'Sports')]
fig = px.histogram(genre_data, x=genre_data['user_score'], color='genre', barmode='overlay')
fig.show()

Again, the dataset appears to be skewed. We'll set the appropiate parameters for the test based on the Levene test results using median as the center point.

In [30]:
sample_1 = filtered_years[filtered_years['genre'] == 'Action']['user_score'].dropna()
sample_2 = filtered_years[filtered_years['genre'] == 'Sports']['user_score'].dropna()

print('Levene p-value: ', st.levene(sample_1, sample_2, center='median').pvalue)


Levene p-value:  1.045718795108326e-05


The Levene test p-value is significantly lower than 1%, indicating that there is not enough statistical significance to accept the equality in variance. As such, the equal_var( ) parameter will be set to False. Because the null hypothesis is assuming the average user scores are different, a statistical significance indicating they are different should be a p-value greater than 5%.

In [31]:
alpha = 0.05
results = st.ttest_ind(sample_1, sample_2, equal_var = False)
                             
print('T-test p-value: ', results.pvalue)

# if the p-value is less than alpha, we reject the hypothesis
if results.pvalue < alpha:
    print("We reject the null hypothesis")
else:
    print("We can't reject the null hypothesis")

T-test p-value:  5.989458066467418e-20
We reject the null hypothesis


Based on the p-value from the t-test, we reject the null hypothesis. The sample populations indicate there is not enough statistical significance to suggest user scores between the genres are the same. Intuitively, this makes sense; genre and platform preference are fairly subjective categories and personal preference can certainly bias a user's score. Unless games were reviewed by the same user on multiple platforms, game scores (even same titles released on multiple platforms) are fairly independent from one another.

## Conclusion ##

Based on the analysis conducted above, we may be able to improve profitability of game sales in 2017 by targeting audiences based on platform and game genre. Priority for advertisements should be given to modern generation console platforms, specifically the PlayStation 4 and Xbox One. These platforms are in their nascency and their predecessors (PS3 and X360) were historically the most successful. In terms of game genre, the most successful in almost every region were Shooters, Sports, and Platformers, and thus should also be prioritized.

By exception, advertisement campaigns can avoid titles that received a poor score from professional critics. While high scores showed minimal affect on sales, lower scores (less than 60, or 6) negatively impacted overall sales. 

While these trends are accurate for North America and Europe, if advertisements are regionalized, priorities in platform selection may need to adjust for a campaign in Japan. Neither the PS4 and Xbox One appeared to be profitible in this region, and priority should be given to other platforms, specifically modern generation hand-helds according to historical data.